In [54]:
%matplotlib widget
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import comb as binom_coeff
import scipy.special as spec
from scipy import stats
from sklearn.neighbors import KernelDensity
import vis_utils as vut

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display as disp

# 1. Understanding Bayesian inference

## 1.1. Developing an intuition

The Bayes rule provides a formal (mathematical) description of a type of inference, the act of guessing by means of observation and knowledge. The Bayes rule equates the **posterior** probabilty $p(h \mid D)$ to the product of the **prior** probability ${p(D)}$ and the **likelihood** $p(D \mid h)$, normalized by the **marginal** probability ${p(D)}$:

$$p(h \mid D) = \frac{p(h)p(D \mid h)}{p(D)}$$

For someone who is not used to hearing this statistical jargon, this formula may appear a bit incoherent. But try not to take the way of rote memorization. The intuition behind the rule is crucial and, frankly, quite elegant.If you are like me, you find it hard to think in new and unfamiliar terms. Try replacing some of the words that you are not absolutely comfortable with by more everyday notions. 

For instance, you can swap "probability" with "strength of belief" or "conviction". Note an important distinction between belief _content_ and belief _strength_. When we say that there is _high probability_ that "a coin is fair", we are expressing a _strong conviction_ of in a "belief" that a coin should behave in certain ways. Further, I will refer to belief strength as conviction and to belief content simply as belief. 

Further, you can replace the latin terms "posterior" and "prior" with more intuitive "after-the-fact" and "before-the-fact" (respectively). Thus, you can think of the _prior probability_ as _conviction before the fact_ and the _posterior probability_ as _conviction after the fact_. The "fact" here refers observing some data. Note that conviction after an observation and conviction before the observation are on the opposite sides of the Bayes rule equation, which means that they are related, yet not identical. Observations can change the strength of our beliefs and the Bayes rule describes one way in which this can happen. At this point it is very useful to rearrange the equation in order emphasize that the Bayes rule is an updating procedure:

$$p(h \mid D) =  p(h) \times \frac{p(D \mid h)}{p(D)}$$

The initial and updated convictions differ by a factor that includes the likelihood term in the numerator and the marginal term in the denomenator. What are they? Lets start with the numerator. Likelihood (not to be confused with probability) is a kind of a score for the quality of some belief (i.e. beief accuracy). How do we assess our beliefs? Directly or indirectly, we do it by comparing them to our observations. When a belief is accurate, it can be used to generate guesses that correspond to (our subjective) reality. More observations enable a better assessment. To give you an example, imagine that I am convinced in a belief that "my farts don't smell". Using a sophisticated "cupping" procedure I find out day-to-day that as a matter of fact they are indeed extremely funky. My belief is no good, and I should take it less seriously. But note that if I only sniff my fart once and it happens to stink, I should not be so quick to think that my belief is strictly inaccurate (may be it was a freaky one?). As a result, my conviction in my original belief will not change a lot. So, you can think of likelihood simply as the accuracy of a belief, keeping in mind that the only way to test this accuracy is to compare the belief to reality.

Finally, let's break down the marginal likelihood term $P(D)$ in the denomenator. Mathematically, marginalization of likelihood is a kind of averaging of likelihoods (accuracies) across different beliefs, $p(D) = \int_{H} p(D \vert h)p(h)dh$ (there is an equivalent formulation for a discrete conditional variable in which we take a sum over probability-weighted values). Note that the formula is very similar to the likelihood formula from beore, only now we are considering the entire space of beliefs included in $H$. Therefore, in simpler terms, the marginal likelihood is the combined accuracy of various beliefs, weighted by how strongly we believe in them. Therefore, if we are equally convinced in all of our beliefs about something (this is called a uniform prior), the marginal likelihood is simply the mean accuracy of these beliefs. Incidentally, the accuracy of beliefs that we feel strongly above will inlfuence the average accuracy a lot more. Try and think of different combinations of beliefs' accuracies and their strengths and how these factors determine the magnitude of the denominator.

As you can see, the denominator combines accuracy in a belief with its strength. It is useful to think of what this product of accuracy and conviction ($p(D \vert h) \times p(h)$) stands for. I find it helpful to think about it as a conviction discounted by accuracy (because the ultimate result of the Bayes rule is probability, which we interpret as belief strength). Actually, likelihood can be greater than 1, since it is a product of probabiliy _densities_ (!), not probabilities. Thus, even the term accuracy is a bit misleading, because we usually think of accuracy as a normalized quantity (i.e. a percentage). It would be more accurate (no pun intended) to say that likelihood is a measure of the amount of evidence for a certain belief. It is unbounded and the more positive data you have, the more evidence you have for a belief. However, it can never be negative because we cannot have a negative density. So, a more sensical way to formulate likelihood is the amount of evidence for a belief.

Having been introduced to each of the terms in the Bayes rule, we now should be able to think about the it in a very intuitive way. It should be clear how this equation describes a situation where:

> I am aware of various beliefs about the world in which I can be more or less convinced. Experiencing new things may change how strongly I feel about my beliefs. How? Well, if a belief corresponds well with my observations, I should be more convinced in its truth. On the other hand, if there there are more accurate alternative beliefs that fit my observations better, I should be less convinced aboutmy original less accurate belief. That is all there is to it. Changing convictions in my beliefs before-the-fact requires me know to two things: (1) how accurate my belief is against what I observed, and (2) how accurate are other beliefs against what I observed.

So, the right-hand side of the Bayes rule simply tells us how the data updates an old belief $p(h)$ into a new belief $p(h \mid D)$. If we rearrange it slightly (as above), it becomes very clear that the strength of the initial belief should grow in proportion to its accuracy and shrink in proportion to accuracies of other beliefs:

$$\text{updated conviction in belief}= \text{initial conviction in belief} \times \frac{\text{accuracy of initial belief}}{\text{accuracy of alternative beliefs}}$$

The Bayes rule is super fun because there are multiple ways of thinking about it. If we expand the denominator and put the prior back into the numerator we get:

$$p(h \mid D) = \frac{p(h)p(D \mid h)}{\int_{H} p(D \vert h)p(h)dh}$$

Remember how we said that the product of accuracy and conviction can be thought of as "discounted conviction", a combination of the amount of evidence and strength of a belief? Well, in this form of the Bayes rule we can see that our conviction in a belief after-the-fact equals the discounted (or upgraded) conviction in a belief before-the-fact, relative to the sum-total of discounted or upgraded convictions in other beliefs. 

Hopefully, this section has provided you with some intuitive understanding of what the Bayes rule stands for. Now let's consider a simple example.

## 1.2. A simple example with discrete beliefs

Statisticians love to talk about unfair coins, even though it is a myth and you can't really bias a coin flip. However, imagine someone gave you a coin and told you that it was biased to come up heads only 30% of the time. This is an extraordinary claim, since you think that a biased coin is nearly impossible. We can formalize the extraordinariness of this belief by assigning a very low probability to the belief that the coin is biased. To do that, we need to express our belief numerically. This can be done through Bernoulli PMF, which has a single parameter $\pi$ that corresponds to bias in a binary event:

$$P(X=x; pi) = \pi^{x} (1 - \pi)^{1-x}$$ 

where $x \in \{0, 1\}$ (0 = tails, 1 = heads). The parameter $\pi$ expresses our belief in a precise numerical manner. If we believe that the coin is fair $\pi$ should be $.5$. It corresponds to the proportion of heads we expect (under our belief) if we tossed the coin indefinitely. A belief that the coin is biased towards heads, we could be written as $\pi = .8$. Importantly, the values of $\pi$ correspond only to belief _content_ and not our convictions about particular beliefs.

Let's say our prior belief that the coin is in fact biased to come up heads only 30% of the time is very low, $p(\pi=.3)=.01$, since we don't think such a coin exists. This means that we are almost sure that the coin is fair, $p(\pi=.5) = .99$. But let's be good scientists and test our beliefs against some actual data. Let's say we flip a coin 5 times and get the following sequence `[1,0,0,0,0]`. How does observing this sequence update our beliefs? The Bayes rule tells us that we need to know the likelihoods of all our beliefs (we have two, $\pi=.3$ and $\pi=.5$). By definition, the likelihood (of independent) multiple outcomes of an event is the product of their probability densities or masses. So, the likelihood that the coin is biased is the product of outputs of the Bernoulli PMF evaluated on each toss:

$$\text{likelihood}(\{1,0,0,0,0\};\pi=.3) = .3^{1} \times (1 - .3)^{4} = 0.36$$

and the likelihood that it is fair is:

$$\text{likelihood}(\{1,0,0,0,0\};\pi=.5) = .5^{1} \times (1 - .5)^{4} = .16$$

So it looks like there is more evidence for the belief that the coin is biased, compared to the belief in a fair coin. So, should we believe more strongly now that the coin is biased? We would if we did not have any strong beliefs about the improbability of a biased coin. Remember, the likelihoods only tell us how the initial convictions change. Since we felt so strongly about the the biased coin being so improbable (very low $p(\pi=.3)$, the conviction in that belief is very strong relative to evidence (it shrinks the corresponding likelihood to 1% of its value):

$p(\pi=.3) \times \text{likelihood}(\{1,0,0,0,0\};\pi=.3) = .01 \times .36 = .0036$

while the conviction in a fair coin barely changes the amount of evidence:

$p(\pi=.5) \times \text{likelihood}(\{1,0,0,0,0\};\pi=.5) = .99 \times .16 = .15$

As a result, our disbelief that the coin is biased barely changes:

$$p(\pi=.3 \mid \{1,0,0,0,0\}) = \frac{.0036}{.15 + .0036} = .02$$

What would happen if we had more evidence? It depends on what this evidence favored. Suppose we tossed the coin 100 times and got 30 heads. Would you be convinced that it was biased, despite not believing so in the beginning? Let's see what a Bayesian would say. Note that multiple independent coin tosses is a binomial process. Like the Bernoulli PMF, the binomial PMF has only one parameter which we can interpret the same way, as the probability of heads on individual coin tosses. The likelihood function is:

$$L(N, k \mid \pi) = \binom{N}{k} \pi^{k} (1 - \pi)^{N-k}$$

where $N$ and $k$ is our data. $N$ stands for the total number of tosses and $k$ stands for the number of heads in a sequence, and $\binom{N}{k}$ is the binomial coefficient. Let's use python to help with calculations:

In [ ]:
def binom_likelihood(N, k, pi):
    B = binom_coeff(N, k) # binomial coefficient
    likelihood = B * pi**k * (1 - pi)**(N - k) # likelihood of data, given pi
    return likelihood

# Beliefs and their strengths before data
coin_biased = .01
coin_fair = .99

# Data
N = 100
k = 30

# Updating
evidence_biased = binom_likelihood(N, k, pi=.3)
evidence_fair = binom_likelihood(N, k, pi=.5)

new_coin_biased = (evidence_biased*coin_biased)/sum([evidence_fair*coin_fair, evidence_biased*coin_biased])

print('Posterior belief that coin is 30%-heads-biased = {:.4}'.format(new_coin_biased))

Intuitively, is the data of 30/100 heads more convincing (that the coin is biased) compared to 1/5 heads? It is to me, and it is in line with the Bayes rule as well.

## 1.3. Another simple example with continuous beliefs

Now, suppose you were not so sure that a biased coin is a statistician's fiction. You could, for example, believe that coins can be biased in any particular way and while _most_ coins are fair some are biased towards heads and some are biased towards tails. Whereas before, we formalized our belief as a discrete distribution over 2 parameter values ($\pi \in \{.3, .5\})$, our new continuous belief system does not constrain the value of $\pi$ beyond the possible boundaries, so that $\pi$ could be any real number within its bounds: $0 \leq \pi \in \mathbb{R} \leq 1$. This set of beliefs can be expressed by any bounded continuous distribution. We will use a 2-parameter Beta-distribution, as it allows us to express a higher likelihood for some values of $\pi$ (e.g. that fair coins are more common). In fact, beta-distribution is extremely versatile in expressing various belief spaces. As both parameters increase, the distribution starts to peak (expressing strong beliefs about a narrow range of parameter values. If both are 1, the distribution becomes completely flat, corresponding to a uniform conviction across beliefs. When both approach 0, the distribution approaches the Bernoulli distribution. You can play with the beta-distribution below and think of what kinds of belief spaces differently-shaped distributions correspond to.

In [ ]:
def plot_beta(a, b, pi, ax, resolution):
    x = np.linspace(0, 1, num=resolution)
    y = stats.beta.pdf(x, a, b)
    l, u = pi

    
    ax.get_lines().pop().remove()
    ax.add_line(mpl.lines.Line2D(x, y))
    
    ax.findobj(match=mpl.collections.PolyCollection)[0].remove()
    ax.fill_between(x[l:u+1], y[l:u+1], color='b', alpha=.2)
    p = stats.beta.cdf(x[u], a=a, b=b) - stats.beta.cdf(x[l], a=a, b=b)
    
    ax.get_figure().canvas.draw_idle()
    print('pi = [{:.2f}, {:.2f}], p = {:.4f}'.format(x[l], x[u], p))

plt.close()
fig = plt.figure('Interactive beta')
ax = vut.pretty(fig.add_subplot(111))
ax.axvline(.5, color='k', ls='--')
reso = 200

x = np.linspace(0, 1, reso)
y = stats.beta.pdf(x, 10, 10)
l, = ax.plot(x, y)
z = ax.fill_between(x, y, color='b', alpha=.2)
ax.set_ylim(-.05, 6.2)
ax.set_xlim(0,1)
ax.set_xlabel('$\pi$')
ax.set_ylabel('$f(\pi)$')

a_slide = widgets.FloatSlider(min=0.0, max=30., value = 10, step=.05, layout=widgets.Layout(width='80%'))
b_slide = widgets.FloatSlider(min=0.0, max=30., value = 10, step=.05, layout=widgets.Layout(width='80%'))
pi_slide = widgets.IntRangeSlider(value=[0, reso-1], min=0, max=reso-1, step=1, layout=widgets.Layout(width='50%'), readout=False)

interactive_beta = interact(plot_beta, 
                            a=a_slide, b=b_slide, pi=pi_slide,
                            ax=fixed(ax), resolution=fixed(reso))

A continuous distribution is an efficient way to express (infinitely) many beliefs numerically. Instead of listing all of the possible beliefs, which would take forever, we simply define a PDF that maps any sensible belief that we can think of to probability density. Note that the _probability_ of any particular value of $\pi$ is 0, because the probability of a continuous variable is defined as an integral over some range (or the area under the curve). Thus, we no longer assign probabilities to point values. I think this is a more appropriate description of our beliefs about a coin. Think about what we really mean when we believe that a coin is fair? We do not expect it to come up heads _exactly_ 50% of the time, do we? In fact, I would never bet that in a 1000 coin flips, the number of heads will be exactly 500, even if I was absolutely sure that the coin was fair. However, if I could bet that the number of heads would be between, say, 40% and 50% in a 1000, I would be confident in that result. Continuous beliefs are noisy beliefs and there is always some uncertainty associated with them, which is why we don't want to bet on an exact value of a single belief, but instead choose regions of values around that belief.

In the previous example we had a discrete belief space with only 2 beliefs, each with known probabilities, so it was easy to express our convictions numerically. In a continuous case, we assign probabilities to regions of values instead. The `pi` slider in the figure above allows you to select a region of $\pi$ values and shows you the area under the curve of the selected region. Move the $\pi$ sliders to see, given $a = b = 10$, how confident would we be that the coin comes up heads between 40% and 60%? We would be 64.24% confident. What if we had a different distribution of beliefs, like $a = b = 30$? In this case, we would be almost 90% confident. 

So, let's say we start with this kind of prior belief space $a = b = 30$. How then, could we rationally update our beliefs after seeing 30/100 heads? Bayes rule tells us that we need to know 3 quantities in order to derive the posterior. The derivation of these quantities is particularly hairy (at least for my level), but smarter people have done it:

1. Prior distribution (this is our initial, beta distribution):
$$\text{prior} = \text{PDF}_{beta} = f(\pi; a,b) = \frac{\Gamma(a+b)}{\Gamma(a) \Gamma(b)}  \pi^{a-1}(1-\pi)^{b-1}$$

2. Likelihood (the same likelihood as from the previous example):
$$\text{likelihood} = \text{L}(N, k \mid \pi) = \binom{N}{k} \pi^{k} (1 - \pi)^{N-k}$$

3. Marginal likelihood (the integral of the product of the prior and the likelihood is another beta distribution):
$$\text{marginal likelihood} = \int_0^1 \text{L}(N, k \mid \pi)f(\pi; a, b) d \pi \\
= \text{PMF}_{\text{beta-binomial}} = p(N,k; a, b) = \binom{N}{k} \frac{B(k+a, N-k+b)}{B(a,b)}\\
$$

(where $B(a,b) = \frac{\Gamma(a)\Gamma(b)}{\Gamma(a+b)}$, where $\Gamma(x) = \int_0^{\infty} z^{x-1}e^{-z} dz$) are combined to give us the update formula:

$$ p(\pi \mid D=\{N, k\}) = \frac{\Gamma(a+b+N)}{\Gamma(a+k) \Gamma(b+N-k)}  \pi^{a+k-1}(1-\pi)^{b+N-k-1}$$

Note that this is our original (prior) beta distribution with updated parameters:

$$ p(\pi \mid D=\{N, k\}) = f(\pi; a+k,b+N-k)$$

Below you can visualize this update and see how different priors and data interact with each other to update the posterior. Note how the posterior becomes more pointy as we have more data (more certain beliefs). Also note that the posterior is influenced by the prior a lot more when there is not much data. Before the data, we expected (with about 90% certainty) that the coin will come up heads between between 40% and 60% of the time. After seeing the data, we only believe in this result with about 28% probability. Rather, we are 90% certain that the coin will come up heads between 30% and 45% of the time.

In [ ]:
def plot_update(a, b, N, k, pi1, pi2, ax, resolution):
    k_ = round(k*N, 1)
    print('Data: {} heads in {} tosses'.format(int(round(k*N, 1)), int(N)))
    x = np.linspace(0, 1, num=resolution)
    y = stats.beta.pdf(x, a, b)
    y_new = stats.beta.pdf(x, a=a+k_, b=b+N-k_)
    
    ax.get_lines().pop().remove()
    ax.get_lines().pop().remove()
    
    ax.add_line(mpl.lines.Line2D(x, y))
    ax.add_line(mpl.lines.Line2D(x, y_new, color='darkred'))
    
    for poly in ax.findobj(match=mpl.collections.PolyCollection): poly.remove()
    l, u = pi1    
    ax.fill_between(x[l:u+1], y[l:u+1], color='b', alpha=.2)
    p = stats.beta.cdf(x[u], a=a, b=b) - stats.beta.cdf(x[l], a=a, b=b)
    print('Prior pi = [{:.2f}, {:.2f}], p = {:.4f}'.format(x[l], x[u], p))
    
    l, u = pi2
    ax.fill_between(x[l:u+1], y_new[l:u+1], color='r', alpha=.2)
    p = stats.beta.cdf(x[u], a=a+k_, b=b+N-k_) - stats.beta.cdf(x[l], a=a+k_, b=b+N-k_)
    print('Posterior pi = [{:.2f}, {:.2f}], p = {:.4f}'.format(x[l], x[u], p))
    
    ax.get_figure().canvas.draw_idle()

plt.close()
fig = plt.figure('Bayesian update')
ax = vut.pretty(fig.add_subplot(111))
ax.axvline(.5, color='k', ls='--')
reso = 200

x = np.linspace(0, 1, reso)
y = stats.beta.pdf(x, 10, 10)
l, = ax.plot(x, y)
z = ax.fill_between(x, y, color='b', alpha=.2)
ax.set_ylim(-.05, 12.2)
ax.set_xlim(0,1)
ax.set_xlabel('$\pi$')
ax.set_ylabel('$f(\pi)$')

a_slide = widgets.FloatSlider(min=0.0, max=30., value = 10, step=.05, layout=widgets.Layout(width='80%'))
b_slide = widgets.FloatSlider(min=0.0, max=30., value = 10, step=.05, layout=widgets.Layout(width='80%'))
pi_slide1 = widgets.IntRangeSlider(value=[0, reso-1], min=0, max=reso-1, step=1, layout=widgets.Layout(width='50%'), readout=False, description='Prior int.')
pi_slide2 = widgets.IntRangeSlider(value=[0, reso-1], min=0, max=reso-1, step=1, layout=widgets.Layout(width='50%'), readout=False, description='Posterior int.')

N_box = widgets.FloatText(value=100, description='N:')
k_slide = widgets.FloatSlider(min=0, max=1., value = .3, step=.01, layout=widgets.Layout(width='80%'), description='k')

interactive_update = interact(plot_update, 
                            a=a_slide, b=b_slide, pi1=pi_slide1, pi2=pi_slide2, N=N_box, k=k_slide,
                            ax=fixed(ax), resolution=fixed(reso))

# 2. Estimating the posterior

Did you notice how I avoided hand-derivation of the marginal likelihood towards the end of the previous section? This is because I suck at maths. However, sometimes even smart people who actually know maths think that it's just too much to integrate the product of likelihood and posterior in the denominator. Luckily, thanks to modern computer (super) powers, there are ways around this problem.

## 2.1. Markov Chain Monte Carlo
Markov Chain Monter Carlo (MCMC) is a sampling method that allows to characterize a distribution without knowing its mathematical properties by randomly sampling values from the distribution. MCMC can be broken down into two basic components:
- **Monte Carlo** is a practice of estimating the properties of distribution by random sampling
- **Markov Chain** is a Markovian process in which the samples are generated sequentially, such that a sample drawn on step 1 influences the sample drawn on step 2, but no further. The final sample of data is thus determined only by its predecessor sample, but the whole process is a Markov chain

MCMC is used in Bayesian analyses because it allows to approximate aspects of the posterior that are not easily obtained by analytic derivation. One version of MCMC that is used to illustrate the concept of iterative search is the Metropolis algorithm illustrated below. We begin by generating a random coin (a coin with an unknown bias). We toss it a bunch of times and then try to approximate the coin's true bias parameter using the Metropolis search:

In [67]:
plt.close()
fig = plt.figure('Metropolis search', figsize=[12,4])
ax1 = vut.pretty(fig.add_subplot(131))
ax2 = vut.pretty(fig.add_subplot(132))
ax3 = vut.pretty(fig.add_subplot(133))

# Get biased coin, toss it N times, and calculate the proportion of heads:
pi = .30
N = 1000

sample = stats.bernoulli(pi).rvs(size=N)
M = sample.mean()
SD = sample.std()

# ================== METROPOLIS SEARCH ==================
# Generate the proposal distribution (should be symmetric and centered on 0)
noise_distr = stats.norm(loc=0, scale=.002)

# Make a reasonable guess (initial proposal)
guess = .5

# For J iterations generate new proposals
J = 1000
history = []
likes = []
L = stats.binom(n=sample.size, p=M).pmf(k=round(guess*sample.size))
for j in range(J):
    noise = noise_distr.rvs(1)[0]
    new_guess = guess + noise
    if new_guess < 0 or new_guess > 1: continue
    L_guess = stats.binom(n=sample.size, p=M).pmf(k=round(new_guess*sample.size))

    if L_guess >= L:
        L = L_guess
        guess = new_guess
    else:
        accept_new = int(stats.bernoulli(p=L_guess/L).rvs(1))
        L = L_guess if accept_new else L
        guess = new_guess if accept_new else guess
    history.append(guess)

ax1.plot(history, 'green', ls='', marker='o', markersize=2, alpha=.5, label='Metropolis guess')
ax1.axhline(pi, label='True $\pi$', lw=4, color='darkred')
ax1.legend()
ax1.set_ylim(.23,.52)

ax2.plot(stats.binom(n=N, p=pi).pmf(k=np.arange(N)), np.arange(N)/N, c='darkred', label='True PMF')
ax2.legend()
ax2.set_ylim(.23,.52)

kde = stats.gaussian_kde(history, bw_method=.02 / np.std(history, ddof=1)).evaluate(np.arange(N)/N)
ax3.plot(kde, np.arange(N)/N, c='green', label='Sampled distribution')
ax3.hist(history, density=True, orientation='horizontal', bins=50, alpha=.5, color='green')
ax3.set_ylim(.23,.52)
ax3.legend()
fig.tight_layout()

print(pi, guess)

FigureCanvasNbAgg()

0.3 0.3024036723303051


We begin the Metropolis search by choosing an initial parameter value .5. We then evaluate the likelihood of that parameter against the data that we have. After, we make a new guess that is similar to the previous guess and look at its likelihood against the same data. If the new guess is more likely, we accept it. If not, we accept it stochastically. and the change of accepting it is proportional to how likely the new guess is relative to the old guess. E.g. if the likelihood of a new guess is L, and that of the previous guess is K, such that L < K, then the probability of accepting the new guess is L/K.